In [1]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 9.1MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 17.1MB/s 
     |████████████████████████████████| 1.7MB 18.8MB/s 
     |████████████████████████████████| 7.2MB 45.9MB/s 
     |████████████████████████████████| 3.0MB 51.2MB/s 
     |████████████████████████████████| 317kB 51.6MB/s 
     |████████████████████████████████| 890kB 50.1MB/s 
     |████████████████████████████████| 1.1MB 46.9MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 122kB 52.5MB/s 
     |████████████████████████████████| 163kB 54.0MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 133kB 50.7MB/s 
     |████████████████████████████████| 6.7MB 46.1MB/s 
     |████████████████████████████████| 4.4MB 53.2MB/s 
     |████████████████████████████████| 112kB 55.4MB/s 
     |████████████████████████████████| 71kB 10.0MB

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
TRAIN_FILE = "/content/drive/My Drive/project/dataset/model_data_train.csv"
TEST_FILE = "/content/drive/My Drive/project/dataset/model_data_test.csv"

In [2]:
!nvidia-smi

Tue Oct  6 16:53:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!wandb login 09d80773c7d82c79bc5968159fd84c2a6c45169c

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import pandas as pd
import logging
from simpletransformers.classification import ClassificationModel, ClassificationArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
train_df = pd.read_csv(TRAIN_FILE)
test_df = pd.read_csv(TEST_FILE)

In [5]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 30
model_args.regression = True

model_args.overwrite_output_dir=True

model_args.evaluate_each_epoch=True
model_args.evaluate_during_training=True
model_args.evaluate_during_training_verbose=True
model_args.evaluate_during_training_steps=0

model_args.train_batch_size = 64
model_args.learning_rate=3e-6

model_args.output_dir="/content/drive/My Drive/project/model_checkpoints/"

model_args.wandb_project="roberta-simpletransformers"

model = ClassificationModel("roberta", "roberta-base", num_labels=1, args=model_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [8]:
model.train_model(train_df, eval_df=test_df)

KeyboardInterrupt: ignored

In [7]:
model = ClassificationModel("roberta", "/content/drive/My Drive/project/model_checkpoints/checkpoint-2704-epoch-16", num_labels=1, args=model_args)

In [8]:
d = model.eval_model(test_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'eval_loss': 3119719.155001849}


In [9]:
d[1]  

array([-12.9375   ,  13.078125 , -12.7890625, ..., -13.0625   ,
        13.0859375,  13.078125 ])

In [18]:
for i in range(d[1].shape[0]):
  print("Model predicted: " + str(d[1][i]) +" actual value: " + str(test_df.iloc[i, 1]))

Model predicted: -12.9375 actual value: -294.9999999999342
Model predicted: 13.078125 actual value: -524.9999999998867
Model predicted: -12.7890625 actual value: -190.00000000013452
Model predicted: -12.859375 actual value: 205.0000000002328
Model predicted: -12.859375 actual value: 125.00000000015275
Model predicted: -13.046875 actual value: 425.0000000001197
Model predicted: -13.0859375 actual value: 634.9999999997192
Model predicted: 13.0 actual value: -720.0000000000539
Model predicted: 12.96875 actual value: -479.99999999981384
Model predicted: -13.0703125 actual value: -369.9999999999815
Model predicted: 13.0 actual value: 390.00000000011255
Model predicted: 13.078125 actual value: 3174.999999999928
Model predicted: -12.984375 actual value: -1284.9999999999807
Model predicted: -13.0859375 actual value: -2565.0000000001505
Model predicted: -13.0703125 actual value: 985.0000000000135
Model predicted: 12.7265625 actual value: -540.000000000207
Model predicted: 13.078125 actual value